# Refinitiv Data Library for Python
## Content - Pricing - Chain + Streaming Constituents

This notebook demonstrates how to use the Pricing interface to 
- retrieve the constituents of a Chain instrument
- and then request Streaming data for the Constituents themselves

## Import the library and load credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook (which I have done)

In [2]:
open_session()

### Define callbacks to handle streaming data
We will use these later to display the streaming data for the Constituent RICs

In [3]:
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

## Get the Chain's constituent RICs

In [4]:
# Define a Chain object for the Dow Jones Index
dow = pricing.chain.Definition(universe="0#.DJI").get_stream(session=session)
# We just want a snapshot of the current Index
dow.open(with_updates=False)
## Extract the consituent RICs and display
dow_constituents = dow.get_constituents()
print(dow_constituents)

['AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']


## Create Streams for all the consitutuents and specify callbacks to display them

In [5]:
# Define our Streaming Price object for all the consituents
pricing_stream = pricing.Definition(universe=dow_constituents, fields=["BID", "ASK"]).get_stream()
# Specify the callback to create and display a dataframe 
# on_complete is invoked once we receive the initial image for all our instruments
pricing_stream.on_complete(create_dataframe)
#Specify the callback to display subsequent updates as and when received
pricing_stream.on_update(update_dataframe)


## Open the streams for all our constituents

In [6]:
# Request all instrument from server and open streams
pricing_stream.open()
# The callbacks we declared earlier should be invoked as and when we receive server responses
# i.e. create_dataframe will be called once we have the initial values for each instrument
# after which, update_dataframe will be invoked as and when any instrument's BID/ASK fields are updated

,BID,ASK
AAPL.OQ,129.79,129.8
AMGN.OQ,239.46,239.49
AXP.N,165.22,165.24
BA.N,245.39,245.45
CAT.N,216.62,216.71
CRM.N,244.3,244.41
CSCO.OQ,53.85,53.86
CVX.N,109.56,109.57
DIS.N,176.91,176.95
DOW.N,66.63,66.66


## Close the streams for the constituents

In [7]:
pricing_stream.close()

<StreamState.Closed: 1>

## Close the session

In [8]:
close_session()